<a href="https://colab.research.google.com/github/Salka18/Integration-de-donn-es-connect-e/blob/salka18/Premiere_pas_with_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Spark

In [1]:
# !pip install --user pyspark (colab)
# install Java JDK 8
# install SPARK
# install pyspark
# eventually consider pyspark
# eventuelly consider SPARK_HOME, or SPARK_PATH, etc.

In [ ]:
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home'

In [4]:
! pip install  pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 61.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=f7dfef95b81982eedd51866557e36ac1c22d8fb261fcb4bf62b376ff363afba8
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [5]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName("Python Spark").getOrCreate()

# Loading the data

In [7]:
df_ratings = spark.read\
    .option("delimiter", "\t")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv('u.data')

In [8]:
# print the dataframe schema
df_ratings.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [9]:
# show a sample of the data (the dataframe executes the whole pipeline at this stage)
df_ratings.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [10]:
# set the rdd equivalent of the dataframe
rdd_ratings = df_ratings.rdd

# Basic Queries

#### Exercice 1 - Number of movies per user (using RDD then Dataframe)

Calculer pour chaque utilisateur le nombre de films notés, et afficher le résultat pour l'un d'entre eux. Utilisez dans un premier temps les RDD puis les Dataframes.

In [11]:
# classical RDD approach
result_1 = rdd_ratings.map(lambda t: (t[0], 1)).reduceByKey(lambda v1, v2: v1 + v2).take(1)
print(result_1)

[(196, 39)]


In [12]:
# dataframe approach, filtering on the previous user to compare results
df_ratings.groupBy('user_id')\
    .count()\
    .filter(df_ratings['user_id']==result_1[0][0])\
    .show(1)

+-------+-----+
|user_id|count|
+-------+-----+
|    196|   39|
+-------+-----+



#### Exercice 2 - Average rating per user (using RDD then Dataframe)

Calculer pour chaque utilisateurs la note moyenne donnée et afficher le résultat pour l'un d'entre eux. Utilisez dans un premier temps les RDD puis les Dataframes.

In [13]:
# classical RDD approach
rdd_map = rdd_ratings.map(lambda t: (t[0], int(t[2])))
rdd_agg = rdd_map.aggregateByKey(
    (0, 0), 
    lambda x,y: (x[0] + y,    x[1] + 1), 
    lambda a,b: (x[0] + y[0], x[1] + y[1])
)

rdd_result = rdd_agg.mapValues(lambda v: float(v[0])/v[1])

result_1 = rdd_result.take(1)
print(result_1)

[(196, 3.6153846153846154)]


In [27]:
# dataframe approach
from pyspark.sql.functions import avg, max

df_ratings.groupBy('user_id')\
    .agg(avg('rating').alias('moyenne'), max('rating').alias('maximum'))\
    .filter(df_ratings['user_id']==result_1[0][0])\
    .show(1)

+-------+------------------+-------+
|user_id|           moyenne|maximum|
+-------+------------------+-------+
|    196|3.6153846153846154|      5|
+-------+------------------+-------+



#### Exercice 3 - Top-5 movies with at least 15 votes (Dataframe)

Afficher les 5 meilleurs films parmi ceux qui ont reçu au moins 15 votes.

*Indices:*
* Utiliser df_ratings pour calculer la moyenne, filtrer les films qui ont moins de 15 notes et classer les films par ordre décroissant.
* Faire un join avec df_movies pour afficher le nom des films sélectionnés.

In [28]:
df_items = spark.read\
    .option("delimiter", "|")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv('u.item')

In [36]:
df_ratings.groupBy('user_id')\
  .agg(count('rating').alias('thiokkou'),avg('rating').alias('banaya'))\
  .show(5)

+-------+--------+------------------+
|user_id|thiokkou|            banaya|
+-------+--------+------------------+
|    148|      65|               4.0|
|    463|     133|2.8646616541353382|
|    471|      31|3.3870967741935485|
|    496|     129|3.0310077519379846|
|    833|     267| 3.056179775280899|
+-------+--------+------------------+
only showing top 5 rows



In [17]:
from pyspark.sql.functions import avg, count, col

df_gb = df_ratings.groupBy('item_id')\
    .agg(avg('rating'), count('item_id').alias('count'))

In [18]:
df_gb = df_gb.filter(df_gb['count'] >= 15)  # same as filter('count>15')

In [19]:
# join with actual movie features
df_join = df_gb.join(df_items, df_gb['item_id']==df_items['movie_id'])

In [20]:
df_join.select(col("movie_title"), col("avg(rating)"))\
    .sort("avg(rating)", ascending=False)\
    .show(5)  # Java 8 (does not work with Java 12)

+--------------------+-----------------+
|         movie_title|      avg(rating)|
+--------------------+-----------------+
|Close Shave, A (1...|4.491071428571429|
|Schindler's List ...|4.466442953020135|
|Wrong Trousers, T...|4.466101694915254|
|   Casablanca (1942)| 4.45679012345679|
|Wallace & Gromit:...|4.447761194029851|
+--------------------+-----------------+
only showing top 5 rows

